##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# TensorFlow Hub によるテキストの分類: 映画レビュー

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/text_classification_with_hub"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">	GitHub でソースを表示</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>
</td>
  <td>     <a href="https://tfhub.dev/s?module-type=text-embedding"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">TF Hub モデルを参照</a>   </td>
</table>

このノートブックでは、映画レビューのテキストを使用して、レビューを*肯定的評価*と*否定的評価*に分類します。これは、機械学習の問題で広く適用されている、重要な分類手法である*二項*分類の例です。

このチュートリアルでは、TensorFlow Hub と Keras を使用した転移学習の基本アプリケーションを実演します。

ここでは、[Internet Movie Database](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) から抽出した 50,000 件の映画レビューを含む、[大規模なレビューデータセット](https://www.imdb.com/)を使います。レビューはトレーニング用とテスト用に 25,000 件ずつに分割されています。トレーニング用とテスト用のデータは*均衡*しています。言い換えると、それぞれが同数の肯定的及び否定的なレビューを含んでいます。

このノートブックでは、TensorFlow でモデルを構築してトレーニングするための [`tf.keras`](https://www.tensorflow.org/guide/keras) という高レベル API と、トレーニング済みモデルを [TFHub](https://tfhub.dev) から 1 行のコードで読み込むためのライブラリである [`tensorflow_hub`](https://www.tensorflow.org/hub) を使用します。`tf.keras` を使用した、より高度なテキスト分類チュートリアルについては、<a>MLCC テキスト分類ガイド</a> をご覧ください。

In [ ]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

In [ ]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

## IMDB データセットをダウンロードする

IMDB データセットは、[imdb reviews](https://www.tensorflow.org/datasets/catalog/imdb_reviews) または [TensorFlow データセット](https://www.tensorflow.org/datasets)で提供されています。次のコードを使って、IMDB データセットをマシン（または Colab ランタイム）にダウンロードしてください。

In [ ]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

## データを確認する

データの形式を確認してみましょう。各サンプルは、映画レビューを表す文章と対応するラベルです。文章はまったく事前処理されていません。ラベルは 0 または 1 の整数値で、0 は否定的なレビューで 1 は肯定的なレビューを示します。

最初の 10 個のサンプルを出力しましょう。

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

最初の 10 個のラベルも出力しましょう。

In [ ]:
train_labels_batch

## モデルを構築する

ニューラルネットワークは、レイヤーのスタックによって作成されています。これには、次の 3 つのアーキテクチャ上の決定が必要です。

- どのようにテキストを表現するか。
- モデルにはいくつのレイヤーを使用するか。
- 各レイヤーにはいくつの*非表示ユニット*を使用するか。

この例では、入力データは文章で構成されています。予測するラベルは、0 または 1 です。

テキストの表現方法としては、文章を埋め込みベクトルに変換する方法があります。トレーニング済みのテキスト埋め込みを最初のレイヤーとして使用することで、次のような 3 つのメリットを得ることができます。

- テキストの事前処理を心配する必要がない。
- 転移学習を利用できる。
- 埋め込みのサイズは固定されているため、処理しやすい。

この例では、[google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2) という、[TensorFlow Hub](https://tfhub.dev) の **トレーニング済みのテキスト埋め込みモデル** を使用します。

このチュートリアルで使用できる、TFHub からの事前トレーニング済みのテキスト埋め込みは他にも数多くあります。

- [google/nnlm-en-dim128/2](https://tfhub.dev/google/nnlm-en-dim128/2) - [google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2) と同じデータで同じ NNLM アーキテクチャを使用してトレーニングされていますが、埋め込みのサイズが大きくなっています。より大きなサイズの埋め込みはタスクを改善できますが、モデルのトレーニングに時間がかかる場合があります。
- [google/nnlm-en-dim128-with-normalization/2](https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2) - [google/nnlm-en-dim128/2](https://tfhub.dev/google/nnlm-en-dim128/2) と同じですが、句読点の削除など、追加のテキスト正規化があります。これは、タスクのテキストに追加の文字や句読点が含まれている場合に役立ちます。
- [google/universal-sentence-encoder/4](https://tfhub.dev/google/universal-sentence-encoder/4) -  ディープアベレージングネットワーク（DAN）エンコーダーでトレーニングされた 512 次元の埋め込みを生成するはるかに大きなモデルです。

その他、多数のテキスト埋め込みモデルがあります。TFHub でその他の[テキスト埋め込みモデル](https://tfhub.dev/s?module-type=text-embedding)を検索してください。

では始めに、TensorFlow Hub モデルを使用して文章を埋め込む Keras レイヤーを作成し、いくつかの入力サンプルで試してみましょう。入力テキストの長さに関係なく、埋め込みの出力形状は、`(num_examples, embedding_dimension)` であるところに注意してください。

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

今度は、完全なモデルを構築しましょう。

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

レイヤーは順にスタックされて、分類器が構築されます。

1. 最初のレイヤーは、TensorFlow Hub レイヤーです。このレイヤーは文章から埋め込みベクトルにマッピングするために、トレーニング済みの SavedModel を使用します。使用中のモデル ([google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2)) は文章をトークンに分割し、各トークンを埋め込んで、埋め込みを組み合わせます。その結果、次元は `(num_examples, embedding_dimension)` となります。この NNLM モデルの場合、`embedding_dimension` は 50 です。
2. この固定長の出力ベクトルは、16 個の非表示ユニットを持つ完全に接続された（`Dense`）レイヤーを介してパイプ処理されます。
3. 最後のレイヤーは、単一の出力ノードと密に接続されています。

モデルをコンパイルしましょう。

### 損失関数とオプティマイザ

モデルをトレーニングするには、損失関数とオプティマイザが必要です。これは二項分類問題であり、モデルはロジット（線形アクティベーションを持つ単一ユニットレイヤー）を出力するため、`binary_crossentropy` 損失関数を使用します。

これは、損失関数の唯一の選択肢ではありません。たとえば、`mean_squared_error` を使用することもできます。ただし、一般的には、確率を扱うには `binary_crossentropy` の方が適しているといえます。これは、確率分布間、またはこのケースではグランドトゥルース分布と予測間の「距離」を測定するためです。

後で、回帰問題（家の価格を予測するなど）を考察する際に、平均二乗誤差と呼ばれる別の損失関数の使用方法を確認します。

では、オプティマイザと損失関数を使用するようにモデルを構成します。

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## モデルをトレーニングする

モデルを 512 個のサンプルのミニバッチで 10 エポック、トレーニングします。これは、`x_train` と`y_train`テンソルのすべてのサンプルを 10 回イテレーションします。トレーニング中、検証セットの 10,000 個のサンプルで、モデルの損失と正解率を監視します。

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

## モデルを評価する

モデルのパフォーマンスを見てみましょう。2 つの値が返されます。損失（誤差、値が低いほど良）と正確率です。

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

このかなり単純なアプローチで、約 87％ の正解率が達成されます。より高度なアプローチを使えば、95％ に近づくでしょう。

## 参考資料

- 文字列入力を使ったより一般的な方法やトレーニング中の正解率と損失の進捗に関するより詳細な分析については、[事前処理済みのテキストを使用したテキスト分類](./text_classification.ipynb)チュートリアルをご覧ください。
- TFHub のトレーニング済みモデルを使用して、[テキスト関連のチュートリアル](https://www.tensorflow.org/hub/tutorials#text-related-tutorials)を試してみてください。